# Create Questions list

Code to create a list of questions based on census metadata, but it may not work, and it costs about $90 in OpenAI credits to tun
Probably should use the cache file in the data directory. 


In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [3]:
# Just load the compressed data and exit, because we don't want to re-run the OpenAI process. 
df = pd.read_csv('../data/questions.csv.gz', compression='gzip')
display(df.head())
raise SystemExit("Stop!")

,Unnamed: 0.1,Unnamed: 0,column_id,measure,restriction,description,time,place,question
0,0,0,B19037E_015,Age Of Householder By Household Income In The ...,"both sexes, Native Hawaiian, ages 0 to 25",<description>,NaN,NaN,How many Native Hawaiian households with a hou...
1,1,1,B26209_057,Group Quarters Type (5 Types) By Geographical ...,both sexes who are in noninstitutionalized gro...,<description>,in the past year,"in Spalding County, Georgia",How does the geographical mobility in the past...
2,2,2,B19037E_016,Age Of Householder By Household Income In The ...,"both sexes, Native Hawaiian, ages 0 to 25",<description>,in 2020,NaN,What is the average age of householders with a...
3,3,3,B26209_058,Group Quarters Type (5 Types) By Geographical ...,all population who are living in college/unive...,<description>,in the past 12 months,by cities,How many people in the United States were livi...
4,4,4,B26209_059,Group Quarters Type (5 Types) By Geographical ...,both sexes who are living in college/universit...,<description>,NaN,by cities,What percentage of college/university student ...


SystemExit: Stop right there!

/Users/eric/opt/miniconda3/envs/data/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

In [ ]:

# Todo, move this into the metadata. 
pkg = mp.open_package('index:civicknowledge.com-census_meta-2020e5')
paths = pkg.resource('paths').dataframe()
mdf = pkg.resource('metadata').dataframe()
mdf.head()

In [ ]:
with shelve.open('census_questions') as db:
    print(len(db), 'existing records')
    print(len(mdf),'column records')
    tasks = questions_tasks(mdf)

    print(len(tasks),' original tasks')

    tasks = [t for t in tasks if t['column_id'] not in db ]
    
print(len(tasks),' filtered tasks')

shuffle(tasks)

batches = list(chunked(tasks, 5))

print(len(batches), 'batches')

print(json.dumps(batches[0], indent=4))

In [ ]:
def run_batch(batch):
    
    response = write_question(batch)
    
    tokens_used.append(response['usage']['total_tokens'])

    with shelve.open('census_questions') as db:
        store_question_responses(task, response, db)


from openai.error import ServiceUnavailableError
from time import sleep

failures_count = 0

shuffle(tasks)

for task_n, task in enumerate(tqdm(tasks)):
    while True:
        
        if failures_count > 4:
            raise Exception("Too many failures")
          
        failures_count += 1;
        
        try:
            run_batch(task)
            failures_count = 0
            break
        except ServiceUnavailableError as e:       
            print(e)
            sleep(45)
            
        except Exception as e:
            print(f"Batch {task_n} Failed: ", e)
            if failures_count == 3:
                raise
            